Nov 2022

This is a fork of https://github.com/adashofdata/nlp-in-python-tutorial.  Rather than using the transcripts of stand-up comedians, it uses the text of New Testament books of the Bible.

# Topic Modeling

## Introduction

Another popular text analysis technique is called topic modeling. The ultimate goal of topic modeling is to find various topics that are present in your corpus. Each document in the corpus will be made up of at least one topic, if not multiple topics.

In this notebook, we will be covering the steps on how to do **Latent Dirichlet Allocation (LDA)**, which is one of many topic modeling techniques. It was specifically designed for text data.

To use a topic modeling technique, you need to provide (1) a document-term matrix and (2) the number of topics you would like the algorithm to pick up.

Once the topic modeling technique is applied, your job as a human is to interpret the results and see if the mix of words in each topic make sense. If they don't make sense, you can try changing up the number of topics, the terms in the document-term matrix, model parameters, or even try a different model.

## Topic Modeling - Attempt #1 (All Text)

In [1]:
# Let's read in our document-term matrix
import pandas as pd
import pickle

data = pd.read_pickle('dtm_stop.pkl')
data

,aaron,abaddon,abandon,abandoned,abandoning,abandons,abba,abel,abhor,abiathar,...,zealous,zebedee,zebulun,zechariah,zenas,zerah,zerubbabel,zeus,zion,zionhe
Matthew,0,0,0,0,0,0,0,1,0,0,...,0,6,2,1,0,1,2,0,1,0
Mark,0,0,0,0,0,0,1,0,0,1,...,0,4,0,0,0,0,0,0,0,0
Luke,1,0,0,0,0,0,0,1,0,0,...,0,1,0,12,0,0,1,0,0,0
John,0,0,1,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
Acts,1,0,2,3,0,0,0,0,0,0,...,1,0,0,0,0,0,0,2,0,0
Romans,0,0,0,1,0,0,1,0,2,0,...,1,0,0,0,0,0,0,0,1,1
1 Corinthians,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2 Corinthians,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Galatians,0,0,0,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,0,0
Ephesians,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [2]:
# Import the necessary modules for LDA with gensim
# Terminal / Anaconda Navigator: conda install -c conda-forge gensim
from gensim import matutils, models
import scipy.sparse

# import logging
# logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [3]:
# One of the required inputs is a term-document matrix
tdm = data.transpose()
tdm.head()

,Matthew,Mark,Luke,John,Acts,Romans,1 Corinthians,2 Corinthians,Galatians,Ephesians,...,Philemon,Hebrews,James,1 Peter,2 Peter,1 John,2 John,3 John,Jude,Revelation
aaron,0,0,1,0,1,0,0,0,0,0,...,0,3,0,0,0,0,0,0,0,0
abaddon,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
abandon,0,0,0,1,2,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
abandoned,0,0,0,0,3,1,0,1,0,0,...,0,0,0,0,0,0,0,0,2,0
abandoning,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [4]:
# We're going to put the term-document matrix into a new gensim format, from df --> sparse matrix --> gensim corpus
sparse_counts = scipy.sparse.csr_matrix(tdm)
corpus = matutils.Sparse2Corpus(sparse_counts)

In [5]:
# Gensim also requires dictionary of the all terms and their respective location in the term-document matrix
cv = pickle.load(open("cv_stop.pkl", "rb"))
id2word = dict((v, k) for k, v in cv.vocabulary_.items())

Now that we have the corpus (term-document matrix) and id2word (dictionary of location: term), we need to specify two other parameters - the number of topics and the number of passes. Let's start the number of topics at 2, see if the results make sense, and increase the number from there.

In [6]:
# Now that we have the corpus (term-document matrix) and id2word (dictionary of location: term),
# we need to specify two other parameters as well - the number of topics and the number of passes
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=2, passes=10)
lda.print_topics()

[(0,
  '0.004*"earth" + 0.004*"sisters" + 0.004*"body" + 0.004*"glory" + 0.004*"sin" + 0.004*"let" + 0.003*"heaven" + 0.003*"person" + 0.003*"gospel" + 0.003*"death"'),
 (1,
  '0.012*"man" + 0.009*"went" + 0.007*"disciples" + 0.006*"tell" + 0.005*"saw" + 0.005*"saying" + 0.005*"away" + 0.005*"peter" + 0.005*"heard" + 0.005*"sent"')]

In [7]:
# LDA for num_topics = 3
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=3, passes=10)
lda.print_topics()

[(0,
  '0.008*"earth" + 0.006*"body" + 0.005*"angel" + 0.005*"heaven" + 0.005*"seven" + 0.004*"saw" + 0.004*"heard" + 0.004*"beast" + 0.004*"man" + 0.004*"throne"'),
 (1,
  '0.012*"man" + 0.009*"went" + 0.007*"paul" + 0.005*"saw" + 0.005*"saying" + 0.005*"disciples" + 0.005*"began" + 0.005*"peter" + 0.005*"house" + 0.005*"tell"'),
 (2,
  '0.007*"man" + 0.006*"world" + 0.005*"disciples" + 0.005*"sin" + 0.004*"went" + 0.004*"tell" + 0.004*"away" + 0.004*"believe" + 0.004*"replied" + 0.004*"heaven"')]

In [8]:
# LDA for num_topics = 4
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=4, passes=10)
lda.print_topics()

[(0,
  '0.010*"world" + 0.008*"man" + 0.007*"sin" + 0.007*"replied" + 0.006*"believe" + 0.005*"disciples" + 0.005*"sent" + 0.005*"going" + 0.004*"went" + 0.004*"person"'),
 (1,
  '0.010*"went" + 0.010*"man" + 0.008*"disciples" + 0.007*"tell" + 0.006*"heaven" + 0.006*"kingdom" + 0.005*"saw" + 0.005*"away" + 0.005*"house" + 0.005*"saying"'),
 (2,
  '0.010*"man" + 0.004*"tell" + 0.003*"went" + 0.003*"sent" + 0.003*"house" + 0.003*"away" + 0.003*"place" + 0.003*"saying" + 0.003*"men" + 0.003*"began"'),
 (3,
  '0.009*"paul" + 0.006*"man" + 0.005*"earth" + 0.005*"heard" + 0.004*"saying" + 0.004*"great" + 0.004*"saw" + 0.004*"heaven" + 0.004*"body" + 0.004*"angel"')]

We've tried modifying our parameters. Let's try modifying our terms list as well.

## Topic Modeling - Attempt #2 (Nouns Only)

One popular trick is to look only at terms that are from one part of speech (only nouns, only adjectives, etc.). Check out the UPenn tag set: https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html.

In [9]:
# Let's create a function to pull out nouns from a string of text
from nltk import word_tokenize, pos_tag

def nouns(text):
    '''Given a string of text, tokenize the text and pull out only the nouns.'''
    is_noun = lambda pos: pos[:2] == 'NN'
    tokenized = word_tokenize(text)
    all_nouns = [word for (word, pos) in pos_tag(tokenized) if is_noun(pos)] 
    return ' '.join(all_nouns)

In [10]:
# Read in the cleaned data, before the CountVectorizer step
data_clean = pd.read_pickle('data_clean.pkl')
data_clean

,book_text
Matthew,this is the record of the genealogy of jesus ...
Mark,the beginning of the gospel of jesus christ t...
Luke,now many have undertaken to compile an accoun...
John,in the beginning was the word and the word wa...
Acts,i wrote the former account theophilus about a...
Romans,from paul a slave of christ jesus called to b...
1 Corinthians,from paul called to be an apostle of christ j...
2 Corinthians,from paul an apostle of christ jesus by the w...
Galatians,from paul an apostle not from men nor by huma...
Ephesians,from paul an apostle of christ jesus by the w...


In [11]:
# # Make sure necessary packages are available
# import nltk
# nltk.donwload('punkt')
# nltk.download('averaged_perceptron_tagger')

In [12]:
# Apply the nouns function to the transcripts to filter only on nouns
data_nouns = pd.DataFrame(data_clean.book_text.apply(nouns))
data_nouns

,book_text
Matthew,record genealogy jesus son david son abraham a...
Mark,beginning gospel jesus son god prophet isaiah ...
Luke,account things accounts eyewitnesses servants ...
John,beginning word word god word word god things t...
Acts,account theophilus jesus day orders holy spiri...
Romans,slave christ jesus set apart gospel god gospel...
1 Corinthians,paul apostle christ jesus god brother church g...
2 Corinthians,apostle christ jesus god brother church god co...
Galatians,men agency christ father brothers churches gal...
Ephesians,apostle christ jesus god saints ephesus faithf...


In [13]:
# Create a new document-term matrix using only nouns
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer

# If we wanted to add any additional stop words, we could do that here
add_stop_words = []

stop_words = text.ENGLISH_STOP_WORDS.union(add_stop_words)

# Recreate a document-term matrix with only nouns
cvn = CountVectorizer(stop_words=stop_words)
data_cvn = cvn.fit_transform(data_nouns.book_text)
data_dtmn = pd.DataFrame(data_cvn.toarray(), columns=cvn.get_feature_names_out())
data_dtmn.index = data_nouns.index
data_dtmn

,aaron,abandons,abba,abel,abhor,abiathar,abijah,abilene,ability,ablaze,...,zeal,zealot,zebedee,zebulun,zechariah,zerah,zerubbabel,zeus,zion,zionhe
Matthew,0,0,0,1,0,0,0,0,1,0,...,0,1,6,2,1,1,2,0,1,0
Mark,0,0,0,0,0,1,0,0,0,0,...,0,1,4,0,0,0,0,0,0,0
Luke,1,0,0,1,0,0,1,1,0,0,...,0,1,1,0,10,0,1,0,0,0
John,0,1,0,0,0,0,0,0,0,0,...,1,0,1,0,0,0,0,0,1,0
Acts,0,0,0,0,0,0,0,0,1,0,...,0,1,0,0,0,0,0,2,0,0
Romans,0,0,1,0,1,0,0,0,0,0,...,2,0,0,0,0,0,0,0,1,1
1 Corinthians,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2 Corinthians,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
Galatians,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Ephesians,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
# Create the gensim corpus
corpusn = matutils.Sparse2Corpus(scipy.sparse.csr_matrix(data_dtmn.transpose()))

# Create the vocabulary dictionary
id2wordn = dict((v, k) for k, v in cvn.vocabulary_.items())

In [15]:
# Let's start with 2 topics
ldan = models.LdaModel(corpus=corpusn, num_topics=2, id2word=id2wordn, passes=10)
ldan.print_topics()

[(0,
  '0.038*"jesus" + 0.030*"god" + 0.018*"lord" + 0.016*"christ" + 0.015*"man" + 0.015*"son" + 0.014*"father" + 0.014*"things" + 0.013*"people" + 0.010*"spirit"'),
 (1,
  '0.023*"god" + 0.014*"paul" + 0.014*"people" + 0.013*"lord" + 0.010*"man" + 0.010*"jesus" + 0.009*"earth" + 0.008*"spirit" + 0.008*"day" + 0.008*"men"')]

In [16]:
# Let's try topics = 3
ldan = models.LdaModel(corpus=corpusn, num_topics=3, id2word=id2wordn, passes=10)
ldan.print_topics()

[(0,
  '0.039*"god" + 0.023*"christ" + 0.023*"jesus" + 0.023*"lord" + 0.015*"things" + 0.011*"spirit" + 0.011*"world" + 0.009*"earth" + 0.009*"people" + 0.009*"father"'),
 (1,
  '0.050*"jesus" + 0.024*"man" + 0.022*"son" + 0.018*"father" + 0.015*"disciples" + 0.015*"people" + 0.014*"god" + 0.011*"things" + 0.011*"lord" + 0.009*"kingdom"'),
 (2,
  '0.035*"god" + 0.019*"lord" + 0.015*"jesus" + 0.015*"christ" + 0.015*"people" + 0.013*"law" + 0.012*"spirit" + 0.011*"paul" + 0.011*"faith" + 0.010*"things"')]

In [17]:
# Let's try 4 topics
ldan = models.LdaModel(corpus=corpusn, num_topics=4, id2word=id2wordn, passes=10)
ldan.print_topics()

[(0,
  '0.002*"jesus" + 0.001*"god" + 0.001*"people" + 0.001*"lord" + 0.001*"son" + 0.001*"father" + 0.001*"things" + 0.001*"man" + 0.001*"christ" + 0.001*"disciples"'),
 (1,
  '0.052*"jesus" + 0.024*"man" + 0.022*"son" + 0.019*"father" + 0.015*"god" + 0.015*"disciples" + 0.015*"people" + 0.012*"things" + 0.012*"lord" + 0.008*"kingdom"'),
 (2,
  '0.023*"god" + 0.022*"earth" + 0.016*"angel" + 0.013*"beast" + 0.012*"throne" + 0.010*"voice" + 0.009*"lamb" + 0.008*"city" + 0.008*"heaven" + 0.008*"people"'),
 (3,
  '0.041*"god" + 0.024*"lord" + 0.024*"christ" + 0.020*"jesus" + 0.014*"spirit" + 0.014*"things" + 0.013*"people" + 0.010*"law" + 0.010*"brothers" + 0.010*"faith"')]

## Topic Modeling - Attempt #3 (Nouns and Adjectives)

In [18]:
# Let's create a function to pull out nouns from a string of text
def nouns_adj(text):
    '''Given a string of text, tokenize the text and pull out only the nouns and adjectives.'''
    is_noun_adj = lambda pos: pos[:2] == 'NN' or pos[:2] == 'JJ'
    tokenized = word_tokenize(text)
    nouns_adj = [word for (word, pos) in pos_tag(tokenized) if is_noun_adj(pos)] 
    return ' '.join(nouns_adj)

In [19]:
# Apply the nouns function to the book texts to filter only on nouns
data_nouns_adj = pd.DataFrame(data_clean.book_text.apply(nouns_adj))
data_nouns_adj

,book_text
Matthew,record genealogy jesus son david son abraham a...
Mark,beginning gospel jesus son god prophet isaiah ...
Luke,many account things accounts eyewitnesses serv...
John,beginning word word god word word god things t...
Acts,i former account theophilus jesus day orders h...
Romans,slave christ jesus apostle set apart gospel go...
1 Corinthians,paul apostle christ jesus god brother church g...
2 Corinthians,apostle christ jesus god brother church god co...
Galatians,apostle men human agency jesus christ father d...
Ephesians,apostle christ jesus god saints ephesus faithf...


In [20]:
# Create a new document-term matrix using only nouns and adjectives, also remove common words with max_df
cvna = CountVectorizer(stop_words=stop_words, max_df=.8)
data_cvna = cvna.fit_transform(data_nouns_adj.book_text)
data_dtmna = pd.DataFrame(data_cvna.toarray(), columns=cvna.get_feature_names_out())
data_dtmna.index = data_nouns_adj.index
data_dtmna

,aaron,abaddon,abandons,abba,abel,abhor,abiathar,abijah,abilene,ability,...,zealot,zealous,zebedee,zebulun,zechariah,zerah,zerubbabel,zeus,zion,zionhe
Matthew,0,0,0,0,1,0,0,1,0,1,...,1,0,6,2,1,1,2,0,1,0
Mark,0,0,0,0,0,0,1,0,0,0,...,1,0,4,0,0,0,0,0,0,0
Luke,1,0,0,0,1,0,0,1,1,0,...,1,0,1,0,11,0,1,0,0,0
John,0,0,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
Acts,0,0,0,0,0,0,0,0,0,1,...,1,1,0,0,0,0,0,2,0,0
Romans,0,0,0,1,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,1,1
1 Corinthians,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2 Corinthians,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Galatians,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
Ephesians,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [21]:
# Create the gensim corpus
corpusna = matutils.Sparse2Corpus(scipy.sparse.csr_matrix(data_dtmna.transpose()))

# Create the vocabulary dictionary
id2wordna = dict((v, k) for k, v in cvna.vocabulary_.items())

In [22]:
# Let's start with 2 topics
ldana = models.LdaModel(corpus=corpusna, num_topics=2, id2word=id2wordna, passes=10)
ldana.print_topics()

[(0,
  '0.011*"world" + 0.011*"law" + 0.010*"man" + 0.008*"earth" + 0.006*"day" + 0.006*"disciples" + 0.005*"heaven" + 0.005*"angel" + 0.005*"death" + 0.005*"flesh"'),
 (1,
  '0.017*"man" + 0.008*"day" + 0.007*"disciples" + 0.007*"house" + 0.007*"paul" + 0.006*"law" + 0.006*"kingdom" + 0.006*"holy" + 0.006*"men" + 0.005*"peter"')]

In [23]:
# Let's try 3 topics
ldana = models.LdaModel(corpus=corpusna, num_topics=3, id2word=id2wordna, passes=10)
ldana.print_topics()

[(0,
  '0.013*"paul" + 0.011*"man" + 0.008*"earth" + 0.008*"day" + 0.007*"holy" + 0.007*"men" + 0.007*"angel" + 0.006*"great" + 0.006*"city" + 0.006*"jews"'),
 (1,
  '0.015*"law" + 0.010*"grace" + 0.008*"gospel" + 0.008*"body" + 0.008*"glory" + 0.007*"righteousness" + 0.006*"sin" + 0.006*"day" + 0.006*"flesh" + 0.006*"world"'),
 (2,
  '0.023*"man" + 0.014*"disciples" + 0.008*"kingdom" + 0.008*"day" + 0.007*"world" + 0.007*"house" + 0.007*"crowd" + 0.007*"law" + 0.006*"heaven" + 0.006*"place"')]

In [24]:
# Let's try 4 topics
ldana = models.LdaModel(corpus=corpusna, num_topics=4, id2word=id2wordna, passes=10)
ldana.print_topics()

[(0,
  '0.012*"body" + 0.009*"grace" + 0.009*"world" + 0.009*"glory" + 0.008*"gospel" + 0.007*"law" + 0.007*"day" + 0.007*"righteousness" + 0.007*"man" + 0.007*"death"'),
 (1,
  '0.015*"earth" + 0.011*"angel" + 0.009*"throne" + 0.008*"beast" + 0.008*"great" + 0.007*"blood" + 0.007*"voice" + 0.007*"heaven" + 0.007*"angels" + 0.006*"lamb"'),
 (2,
  '0.019*"paul" + 0.015*"law" + 0.010*"man" + 0.010*"men" + 0.010*"holy" + 0.009*"jews" + 0.008*"day" + 0.008*"gentiles" + 0.008*"peter" + 0.006*"jerusalem"'),
 (3,
  '0.024*"man" + 0.015*"disciples" + 0.008*"kingdom" + 0.008*"day" + 0.008*"house" + 0.008*"crowd" + 0.007*"law" + 0.007*"world" + 0.007*"place" + 0.007*"john"')]

## Identify Topics in Each Document

Out of the 9 topic models we looked at, the nouns and adjectives, 4 topic one made the most sense. So let's pull that down here and run it through some more iterations to get more fine-tuned topics.

In [25]:
# Our final LDA model (for now)
ldana = models.LdaModel(corpus=corpusna, num_topics=4, id2word=id2wordna, passes=80)
ldana.print_topics()

[(0,
  '0.020*"paul" + 0.014*"man" + 0.011*"men" + 0.011*"day" + 0.010*"holy" + 0.009*"peter" + 0.009*"jews" + 0.007*"days" + 0.007*"house" + 0.006*"jerusalem"'),
 (1,
  '0.020*"man" + 0.012*"disciples" + 0.008*"heaven" + 0.008*"kingdom" + 0.008*"day" + 0.007*"earth" + 0.006*"house" + 0.006*"crowd" + 0.006*"place" + 0.006*"law"'),
 (2,
  '0.019*"law" + 0.009*"grace" + 0.009*"righteousness" + 0.009*"sin" + 0.009*"world" + 0.008*"gospel" + 0.008*"death" + 0.007*"glory" + 0.007*"works" + 0.006*"body"'),
 (3,
  '0.013*"body" + 0.008*"man" + 0.007*"glory" + 0.007*"holy" + 0.007*"day" + 0.007*"grace" + 0.006*"world" + 0.006*"church" + 0.006*"gospel" + 0.005*"spiritual"')]

These four topics look pretty decent. Let's settle on these for now.
* Topic 0: kingdom, heavan, earth
* Topic 1: (man, woman), (husband, wife), (world, church)
* Topic 2: law (righeousness, holy), gospel (grace, glory), sin, death
* Topic 3: paul, peter, disciples, men, jews

In [26]:
# Let's take a look at which topics each book contains
corpus_transformed = ldana[corpusna]
list(zip([max(x)[0] for x in corpus_transformed], data_dtmna.index))

[(1, 'Matthew'),
 (1, 'Mark'),
 (1, 'Luke'),
 (1, 'John'),
 (0, 'Acts'),
 (2, 'Romans'),
 (3, '1 Corinthians'),
 (2, '2 Corinthians'),
 (2, 'Galatians'),
 (3, 'Ephesians'),
 (2, 'Philippians'),
 (3, 'Colossians'),
 (3, '1 Thessalonians'),
 (3, '2 Thessalonians'),
 (2, '1 Timothy'),
 (2, '2 Timothy'),
 (3, 'Titus'),
 (3, 'Philemon'),
 (2, 'Hebrews'),
 (2, 'James'),
 (3, '1 Peter'),
 (3, '2 Peter'),
 (2, '1 John'),
 (2, '2 John'),
 (2, '3 John'),
 (3, 'Jude'),
 (1, 'Revelation')]

For a first pass of LDA:
* Topic 0: kingdom, heavan, earth
* Topic 1: (man, woman), (husband, wife), (world, church)
* Topic 2: law (righeousness, holy), gospel (grace, glory), sin, death
* Topic 3: paul, peter, disciples, men, jews

|Topic 0   |Topic 1|Topic 2      |Topic 3        |
|----------|-------|-------------|---------------|
|Revelation|Matthew|Acts         |Romans         |
|          |Mark   |1 Corinthians|2 Corinthians  |
|          |Luke   |1 Timothy    |Galatians      |
|          |John   |1 Timothy    |Ephesians      |
|          |       |Philemon     |Philippians    |
|          |       |             |Colossians     |
|          |       |             |1 Thessalonians|
|          |       |             |2 Thessalonians|
|          |       |             |Titus          |
|          |       |             |Hebrews        |
|          |       |             |James          |
|          |       |             |1 Peter        |
|          |       |             |2 Peter        |
|          |       |             |1 John         |
|          |       |             |2 John         |
|          |       |             |3 John         |
|          |       |             |Jude           |

## Additional Exercises

1. Try further modifying the parameters of the topic models above and see if you can get better topics.
2. Create a new topic model that includes terms from a different [part of speech](https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html) and see if you can get better topics.